# Detección de puntos de cambio en la series de tiempo

Procedemos a utilizar la Prueba de Razón de Verosimilitud de Worsley (Worsley's Likelihood Ratio Test), que es una técnica para la detección de puntos de cambio en series de tiempo. El objetivo es identificar si hay un cambio significativo en la media o la estructura de la serie en ciertos puntos.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import shutil
import warnings
warnings.filterwarnings('ignore')

lista_zonas = ['MAGDALENA_MEDIO']
path = '../datos/imputados_SARIMA/'

Creamos una funcion para aplicar facilmente la prueba de verosimilitud de Worsley facilmente.

In [31]:
def worsley_statistic(series):
    """
    Calcula la estadística de Worsley para detectar el punto de cambio en la media.
    """
    n = len(series)
    var_total = np.var(series, ddof=1)
    
    worsley_stats = []
    for m in range(1, n):
        mean1 = np.mean(series[:m])
        mean2 = np.mean(series[m:])
        n1 = m
        n2 = n - m
        
        # Evitar la división por cero
        if var_total > 0:
            stat = np.abs((mean1 - mean2) * np.sqrt(n1 * n2 / n))
        else:
            stat = 0
        
        worsley_stats.append(stat)
    
    # Encontrar el punto que maximiza la estadística de Worsley
    max_stat = np.max(worsley_stats)
    max_index = np.argmax(worsley_stats)
    
    return max_stat, max_index

def detect_change_point(series, years):
    """
    Detecta el punto de cambio más significativo usando la estadística de Worsley.
    """
    worsley_stat, change_point = worsley_statistic(series)
    return worsley_stat, years[change_point]

In [5]:
for zona in lista_zonas:
    print('Zona:',zona)
    path_zone = path+zona

    cont = 0
    for file in os.listdir(path_zone):
        print('Estacion:',file)
        df = pd.read_csv(path_zone+'/'+file)
        col = file.split('.')[0]
        

        shutil.rmtree(f'../salidas/{zona}/{col}/worsley/', ignore_errors=True)
        path_output = f'../salidas/{zona}/{col}/worsley/'
        if not os.path.exists(path_output):
            os.makedirs(path_output)
        # Agrupamos por año y encontramos año con el punto de cambio para cada mes

        months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

        change_points = []
        
        for month in months:
            df_month = df[df['Mes'] == month]
            # detectamos el punto de cambio para cada mes
            worsley_stat, change_point_real = detect_change_point(df_month["Precipitación_imputada"].values, df_month["Año"].values)
            change_points.append([month, change_point_real, worsley_stat])

        change_points = pd.DataFrame(change_points, columns=['Mes', 'Punto de Cambio', 'Estadística de Worsley'])
        change_points.to_csv(path_output+'change_points.csv', index=False)

        # # Graficar la serie de tiempo real con el punto de cambio detectado
        # df.set_index('Fecha', inplace=True)
        # plt.figure(figsize=(10, 5))
        # plt.plot(df['Precipitación_imputada'], label='Serie de Tiempo Real')
        # for change_point in change_points['Punto de Cambio']:
        #     plt.axvline(change_point, color='red', linestyle='--', label='Punto de Cambio Detectado')
        # plt.title('Serie de Tiempo Real con el Punto de Cambio Detectado')
        # plt.xlabel('Índice')
        # plt.ylabel('Precipitación_imputada')
        # plt.show()

        # print("Punto de cambio detectado en la serie real:", change_point_real)

Zona: MAGDALENA_MEDIO
Estacion: 23010020.csv
Estacion: 23010080.csv
Estacion: 23020080.csv
Estacion: 23020090.csv
Estacion: 23020100.csv
Estacion: 23025040.csv
Estacion: 23040070.csv
Estacion: 23050080.csv
Estacion: 23050100.csv
Estacion: 23050230.csv
Estacion: 23050250.csv
Estacion: 23055040.csv
Estacion: 23055070.csv
Estacion: 23060110.csv
Estacion: 23060140.csv
Estacion: 23060150.csv
Estacion: 23060160.csv
Estacion: 23060170.csv
Estacion: 23060180.csv
Estacion: 23060190.csv
Estacion: 23060200.csv
Estacion: 23060260.csv
Estacion: 23060290.csv
Estacion: 23060370.csv
Estacion: 23065120.csv
Estacion: 23070010.csv
Estacion: 23070020.csv
Estacion: 23080390.csv
Estacion: 23080640.csv
Estacion: 23080650.csv
Estacion: 23080720.csv
Estacion: 23080740.csv
Estacion: 23080750.csv
Estacion: 23080760.csv
Estacion: 23080810.csv
Estacion: 23080820.csv
Estacion: 23080920.csv
Estacion: 23080940.csv
Estacion: 23085030.csv
Estacion: 23085080.csv
Estacion: 23085110.csv
Estacion: 23085140.csv
Estacion: 23